# 5K
Subsample Voronoi 

In [1]:
import pandas as pd
df = {}

In [2]:
from clustergrammer2 import net

import ipywidgets as widgets
import numpy as np
from bqplot import pyplot as plt
import bqplot

>> clustergrammer2 backend version 0.4.0


In [3]:
from copy import deepcopy
from glob import glob

#### Load X-shift cluster ID

In [4]:
df['cell_type'] = pd.read_excel('../data/big_data/mmc2__codex_cell_type_info.xlsx', index_col=0)
df['cell_type'].shape

(58, 1)

In [5]:
ct_dict = {}
cell_types = []
for inst_id in df['cell_type'].index.tolist():
    inst_ct = df['cell_type']['Imaging phenotype (cell type)'].loc[inst_id]
    cell_types.append(inst_ct)
    ct_dict[inst_id] = inst_ct
    
cell_types = sorted(list(set(cell_types)))
print(len(cell_types))
cell_types    

28


['B cells',
 'B220(+) DN T cells',
 'CD106(+)CD16/32(+)CD31(+) stroma',
 'CD106(+)CD16/32(+)CD31(-)Ly6C(-) stroma',
 'CD106(+)CD16/32(-)Ly6C(+)CD31(+)',
 'CD106(-)CD16/32(+)Ly6C(+)CD31(-)',
 'CD106(-)CD16/32(-)Ly6C(+)CD31(+) stroma',
 'CD11c(+) B cells',
 'CD3(+) other markers (-)',
 'CD31(hi) vascular',
 'CD4(+) T cells',
 'CD4(+)CD8(-)cDC',
 'CD4(+)MHCII(+)',
 'CD4(-)CD8(+)cDC',
 'CD4(-)CD8(-) cDC',
 'CD8(+) T cells',
 'ERTR7(+) stroma',
 'F4/80(+) mphs',
 'FDCs',
 'NK cells',
 'capsule',
 'dirt',
 'erythroblasts',
 'granulocytes',
 'marginal zone mphs',
 'megakaryocytes',
 'noid',
 'plasma cells']

In [6]:
x_dim = 1200
y_dim = 1000

### Expression Data
See http://welikesharingdata.blob.core.windows.net/forshare/index.html

* CSV file contains mean marker intensities and X, Y, Z corrdinates for each cell relative to the top left corner of its tile. 
* Tile IDs correspond to tile images sorted in the alphanumeric order
* Size parameter corresponds to the cell object sizes in voxels
* "imaging phenotype Cluster ID" column specifies the phenotypic cluster identity as determined by X-shift
* "niche Cluster ID" column specifies the i-niche cluster determined by K-means (K=100)
* neighborhood graph files describes which pairs of cells are adjacent to one another

### Add X-Y position ID to the Cells
This ID will be used to generate the neighborhood graph.

In [8]:
%%time
df['exp'] = pd.read_csv('../data/big_data/Suppl.Table2.CODEX_paper_MRLdatasetexpression.csv')

CPU times: user 10.1 s, sys: 520 ms, total: 10.6 s
Wall time: 10.4 s


In [9]:
df['exp'].shape

(734101, 38)

In [10]:
%%time
# new_rows = ['C-' + str(x) for x in df['exp'].index.tolist()]

new_rows = []
for inst_row in df['exp'].index.tolist():
    inst_ser = df['exp'].loc[inst_row]
    inst_x = inst_ser['X.X']
    inst_y = inst_ser['Y.Y']
    inst_tile = inst_ser['sample_Xtile_Ytile']
    
    new_row = 'C-' + str(inst_row) + '_' + inst_tile + '_' + str(inst_x) + '_' + str(inst_y)
    # new_row = inst_tile + '_' + str(inst_x) + '_' + str(inst_y)
    new_rows.append(new_row)

df['exp'].index = new_rows
print(df['exp'].shape)

(734101, 38)
CPU times: user 2min 18s, sys: 396 ms, total: 2min 18s
Wall time: 2min 19s


In [11]:
rows = df['exp'].index.tolist()
print(len(rows))
print(len(list(set(rows))))

734101
734101


### Select Subset of Image Tiles

In [12]:
all_tiles = ['BALBc-1_X01_Y01', 'BALBc-1_X02_Y01', 'BALBc-1_X01_Y02', 'BALBc-1_X02_Y02']

In [13]:
df_list = []


for inst_tile in all_tiles:

    keep_rows = []

    ser_tile = df['exp']['sample_Xtile_Ytile']
    ser_found = ser_tile[ser_tile == inst_tile]
    
    keep_rows.extend(ser_found.index.tolist())
    inst_df = df['exp'].loc[keep_rows].transpose()
    print('inst_df', inst_df.shape)
    
    if 'X02' in inst_tile:
        inst_df.loc['X.X'] = inst_df.loc['X.X'] + 1350

    if 'Y01' in inst_tile:        
        inst_df.loc['Y.Y'] = 2000 - inst_df.loc['Y.Y']
    
    if 'Y02' in inst_tile:
        inst_df.loc['Y.Y'] = 1000 - inst_df.loc['Y.Y'] 
        
    df_list.append(inst_df)
    
    
# df['tile'] = df['exp'].loc[keep_rows].transpose()
df['tile'] = pd.concat(df_list, axis=1)
print(df['tile'].shape)

cats = df['tile'].loc['Imaging phenotype cluster ID']
cats = [ct_dict[x] for x in cats]

new_cols = []

cols = df['tile'].columns.tolist()
for index in range(len(cols)):
    
    new_col = (cols[index], 'Cell Type: ' + str(cats[index]))
    new_cols.append(new_col)
df['tile'].columns = new_cols
df['tile'].shape

inst_df (38, 1127)
inst_df (38, 1247)
inst_df (38, 1481)
inst_df (38, 1291)
(38, 5146)


(38, 5146)

In [18]:
cols = df['exp'].columns.tolist()
exp_cols = cols[1:30]

In [19]:
df['tile'].head()

,"(C-643_BALBc-1_X01_Y01_10_277, Cell Type: B cells)","(C-1264_BALBc-1_X01_Y01_1006_490, Cell Type: B cells)","(C-1589_BALBc-1_X01_Y01_101_504, Cell Type: B cells)","(C-1611_BALBc-1_X01_Y01_101_759, Cell Type: B cells)","(C-1873_BALBc-1_X01_Y01_1013_254, Cell Type: B cells)","(C-1877_BALBc-1_X01_Y01_1013_293, Cell Type: B cells)","(C-1935_BALBc-1_X01_Y01_1013_914, Cell Type: B cells)","(C-2478_BALBc-1_X01_Y01_1020_25, Cell Type: B cells)","(C-2866_BALBc-1_X01_Y01_1025_128, Cell Type: B cells)","(C-2988_BALBc-1_X01_Y01_1026_609, Cell Type: B cells)",...,"(C-250718_BALBc-1_X02_Y02_525_10, Cell Type: NK cells)","(C-251502_BALBc-1_X02_Y02_1008_129, Cell Type: noid)","(C-252813_BALBc-1_X02_Y02_68_787, Cell Type: noid)","(C-252972_BALBc-1_X02_Y02_787_592, Cell Type: noid)","(C-253114_BALBc-1_X02_Y02_869_585, Cell Type: noid)","(C-253125_BALBc-1_X02_Y02_871_770, Cell Type: noid)","(C-253249_BALBc-1_X02_Y02_95_418, Cell Type: noid)","(C-254494_BALBc-1_X02_Y02_399_712, Cell Type: CD4(+)MHCII(+))","(C-254542_BALBc-1_X02_Y02_590_401, Cell Type: CD4(+)MHCII(+))","(C-254598_BALBc-1_X02_Y02_79_540, Cell Type: CD4(+)MHCII(+))"
Imaging phenotype cluster ID,9640,9640,9640,9640,9640,9640,9640,9640,9640,9640,...,9587,9644,9641,9641,9644,9644,9644,9628,9628,9628
CD45,4616.2,1820.99,2748.41,1670.64,1783.57,2547.93,1738.63,615.305,2149.2,440.949,...,2921.56,-1931.85,-434.003,-4177.93,-1177.82,-1901.63,-77.8146,873.133,878.516,-2651.87
Ly6C,109.183,-193.429,1080.91,162.775,-83.4236,-140.051,-193.584,309.844,302.463,-6.71526,...,511.213,-101.499,-108.242,422.338,-175.781,-121.524,83.7457,813.303,-201.251,-81.0684
TCR,56.6081,-253.548,155.409,479.413,980.42,232.764,55.9894,783.197,1934.21,-750.628,...,273.367,-368.264,-1081.77,-3893.62,-136.012,-341.817,-109.301,296.59,-501.445,154.822
Ly6G,16.1121,-108.621,290.299,156.224,79.3494,95.8747,-188.539,135.255,175.713,50.1617,...,124.622,-95.9476,-58.6653,138.927,-161.107,-152.685,41.58,41.3005,-81.5018,-15.0179


In [21]:
df['tile-exp-ini'] = df['tile'].loc[exp_cols]
df['tile-exp-ini'].shape

(29, 5146)

In [22]:
df['tile-exp'] = df['tile-exp-ini']
df['tile-exp'][df['tile-exp'] < 0] = 0

In [23]:
df['tile-exp'][df['tile-exp'] > 5000] = 5000

### Location DAta

In [24]:
df['tile-loc'] = df['tile'].loc[['X.X', 'Y.Y']].transpose()
df['tile-loc'] = df['tile-loc'].astype('int')
df['tile-loc'].shape

(5146, 2)

In [25]:
def set_expression_opacity(inst_gene):
    ser_opacity = df['tile-exp'].loc[inst_gene]
    list_opacity = [float(x/ser_opacity.max()) for x in list(ser_opacity.get_values())]
    scatter.default_opacities = list_opacity

In [26]:
fig = plt.figure(title='Scatter')
def_tt = bqplot.Tooltip(fields=['name'], formats=[''])

scatter = plt.scatter(df['tile-loc']['X.X'], 
                      df['tile-loc']['Y.Y'], 
                      figsize=(20,10), 
                      ylim=(0,1000), 
                      xlim=(0,1000), stroke='black', 
                      tooltip=def_tt, 
                      names=df['tile-loc'].index.tolist(), 
                      display_names=False)

inst_width = 900
fig.layout.min_height = str(inst_width/1.2) + 'px'
fig.layout.min_width = str(inst_width) + 'px'

set_expression_opacity('NKp46')

scatter.default_size = 100
scatter.colors = ['red']

IndexError: list index out of range